In [63]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [65]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [72]:
to_replace = [(constant.UNDEF_VAL, 'most_frequent')]

augment_param = {
    'degree': 5
}

augment_param = None

x_split, y_split = pp.preprocess_jet_num(x, y, to_replace, do_normalise=False, add_bias=True, augment_param=augment_param)

In [73]:
x_split[2].shape, y_split[2].shape

((72543, 30), (72543,))

In [74]:
data_part = []

nb_samples = []
for x_, y_ in zip(x_split, y_split):
    train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
    data = {'train': train_data, 'test': test_data}
    data_part.append(data)
    nb_samples.append(len(test_data[1]))

In [75]:
models = []

learning_param = {
    'max_iters' : 200,
    'gamma': 1e-6,
    'lambda_': 1e-6
}


model_list = ['ridge_regression', 'ridge_regression', 'ridge_regression']
acc_list = []

for idx in range(len(data_part)):
    x_tr, y_tr = data_part[idx]['train']
    nb_features = x_tr.shape[1]
    
    w_initial = np.random.uniform(0,1,size=nb_features) 
    model_ls = model.Model(model_list[idx], w_initial, learning_param, debug=True)
    
    model_ls.learn_weights(y_tr, x_tr)
    models.append(model_ls)
    
    tx_te, y_te = data_part[idx]['test']
    y_ = model_ls.predict(tx_te)
    acc_list.append(misc.accuracy(y_te, y_))

learning weights...
done.
learning weights...
done.
learning weights...
done.


In [76]:
acc_list

[0.8166349714743268, 0.7117616713954088, 0.7245657568238213]

In [71]:
avg_acc = np.dot(acc_list, nb_samples) / sum(nb_samples)
avg_acc

0.7585103404136165